In [1]:
import os
import csv
import random
import numpy as np
import pandas as pd
corpus_list = ['cejc','mpdd']
src_type = 'original'

In [2]:
situation_list = ['apology','request','thanksgiving']
negative_list = ['negative']

if len(situation_list)>1:
    class_type = 'multi'
elif len(situation_list)==1:
    class_type = 'binary'
else:
    import traceback
    try:
        raise Exception
    except:
        traceback.print_exc()
 
sen_type_list = ['query','res']
ver_name = f'001_{class_type}_all_all_prefix'
sen_type_list = ['query']
ver_name = f'011_{class_type}_all_query_prefix'
sen_type_list = ['res']
ver_name = f'021_{class_type}_all_res_prefix'

situation_list = ['apology']
negative_list = ['negative','request','thanksgiving']
sen_type_list = ['query','res']
ver_name = f'101_{class_type}_apology_all_prefix'
sen_type_list = ['query']
ver_name = f'111_{class_type}_apology_query_prefix'
sen_type_list = ['res']
ver_name = f'121_{class_type}_apology_res_prefix'

situation_list = ['request']
negative_list = ['negative','apology','thanksgiving']
sen_type_list = ['query','res']
ver_name = f'201_{class_type}_request_all_prefix'
sen_type_list = ['query']
ver_name = f'211_{class_type}_request_query_prefix'
sen_type_list = ['res']
ver_name = f'221_{class_type}_request_res_prefix'

situation_list = ['request']
negative_list = ['negative','apology','thanksgiving']
sen_type_list = ['query','res']
ver_name = f'301_{class_type}_thanksgiving_all_prefix'
sen_type_list = ['query']
ver_name = f'311_{class_type}_thanksgiving_query_prefix'
sen_type_list = ['res']
ver_name = f'321_{class_type}_thanksgiving_res_prefix'


test_type_list = ['translated','rewrited']
save_dir = f'outputs/situation_classification/{ver_name}/'

In [3]:
def get_data_as_list(path):
    data = []
    with open(path, 'r', encoding='utf-8-sig')as f:
        reader = csv.reader(f)
        for row in reader:
            data.append(row[0])
    return data

def get_df(corpus_list, situation_list, sen_type_list, src_type, train_type, class_type, positive=True):
    target_text = []
    input_text = []
    prefix = []
    for corpus in corpus_list:
        for i, situation in enumerate(situation_list):
            for sen_type in sen_type_list:
                f_path = f'data/{corpus}/{situation}/{src_type}_{sen_type}_{train_type}'
                tmp = get_data_as_list(f_path)
                input_text += tmp
                
                if positive==True:                
                    if class_type=="multi":
                        target_text += [str(i+1)]*len(tmp)
                    elif class_type=="binary":
                        target_text += ['1']*len(tmp)
                elif positive==False: 
                    target_text += ['0']*len(tmp)
                else:
                        try:
                            raise Exception
                        except:
                            traceback.print_exc()
                prefix += [f'{class_type} classification {corpus} {sen_type}']*len(get_data_as_list(f_path))
                
    df = pd.DataFrame([prefix,input_text,target_text], index=['prefix','input_text','target_text']).astype(str).T
    return df

# Finetune

In [ ]:
import logging
import pandas as pd
from simpletransformers.t5 import T5Model, T5Args

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

train_type = 'train'    
train_df = get_df(corpus_list, situation_list, sen_type_list, src_type, train_type, class_type)
train_neg_df = get_df(corpus_list, negative_list, sen_type_list, src_type, train_type, class_type, positive=False)
train_neg_df = train_neg_df.sample(n=len(train_df), random_state=0)
train_df= pd.concat([train_df,train_neg_df])

train_type = 'val'  
eval_df = get_df(corpus_list, situation_list, sen_type_list, src_type, train_type, class_type)
eval_neg_df = get_df(corpus_list, negative_list, sen_type_list, src_type, train_type, class_type, positive=False)
eval_neg_df = eval_neg_df.sample(n=len(eval_df), random_state=0)
eval_df= pd.concat([eval_df,eval_neg_df]) 

# train_df["prefix"] = f'{class_type} classification'
# eval_df["prefix"] = f'{class_type} classification'

display(train_df.iloc[1000:1005])
display(eval_df.iloc[100:105])

In [ ]:
model_args = T5Args()

model_args.max_seq_length = 128
# model_args.length_penalty = 20
model_args.train_batch_size = 4
model_args.eval_batch_size = 4
model_args.num_train_epochs = 20
model_args.evaluate_during_training = True
model_args.evaluate_during_training_steps = 500
model_args.use_multiprocessing = False
model_args.fp16 = False
model_args.early_stopping_metric = 'eval_loss'
model_args.early_stopping_metric_minimize = True
model_args.early_stopping_patience = 3
model_args.use_early_stopping = True
model_args.save_eval_checkpoints = True
model_args.save_eval_checkpoints = False
model_args.learning_rate = 1e-5
model_args.best_model_dir = save_dir+'best_model/'
model_args.output_dir = save_dir+'ckpt/'
model_args.save_model_every_epoch = True
model_args.save_steps = -1
model_args.no_cache = True
model_args.reprocess_input_data = True
model_args.overwrite_output_dir = True
model_args.preprocess_inputs = False
model_args.num_return_sequences = 1
model_args.wandb_project = ver_name

# model = T5Model("mt5", "google/mt5-base", args=model_args)
model = T5Model("mt5", "google/mt5-base", args=model_args, cuda_device=0)
# Train the model
os.environ['WANDB_CONSOLE'] = 'off'
model.train_model(train_df, eval_data=eval_df)

# test

In [23]:
# PLEASE COMMENT OUT WHEN YOU TRAIN AND TEST AT THE SAME TIME

situation_list = ['apology','request','thanksgiving']
negative_list = ['negative']

if len(situation_list)>1:
    class_type = 'multi'
elif len(situation_list)==1:
    class_type = 'binary'
else:
    import traceback
    try:
        raise Exception
    except:
        traceback.print_exc()
 
# sen_type_list = ['query','res']
# ver_name = f'002_{class_type}_all_all_prefix'
# sen_type_list = ['query']
# ver_name = f'012_{class_type}_all_query_prefix'
# # sen_type_list = ['res']
# # ver_name = f'021_{class_type}_all_res_prefix'

situation_list = ['apology']
negative_list = ['request','thanksgiving']
sen_type_list = ['query','res']
ver_name = f'101_{class_type}_apology_all_prefix'
# sen_type_list = ['query']
# ver_name = f'111_{class_type}_apology_query_prefix'
# sen_type_list = ['res']
# ver_name = f'121_{class_type}_apology_res_prefix'

# situation_list = ['request']
# negative_list = ['apology','thanksgiving']
# sen_type_list = ['query','res']
# ver_name = f'201_{class_type}_request_all_prefix'
# sen_type_list = ['query']
# ver_name = f'211_{class_type}_request_query_prefix'
# sen_type_list = ['res']
# ver_name = f'221_{class_type}_request_res_prefix'

# situation_list = ['request']
# negative_list = ['apology','thanksgiving']
# sen_type_list = ['query','res']
# ver_name = f'301_{class_type}_thanksgiving_all_prefix'
# sen_type_list = ['query']
# ver_name = f'311_{class_type}_thanksgiving_query_prefix'
# sen_type_list = ['res']
# ver_name = f'321_{class_type}_thanksgiving_res_prefix'


test_type_list = ['translated','rewrited']
save_dir = f'outputs/situation_classification/{ver_name}/'

In [24]:

import logging
import sacrebleu
import pandas as pd
from simpletransformers.t5 import T5Model, T5Args

test_type_list = ['original','translated','rewrited']
for src_type in test_type_list:
    train_type = 'test'  
    eval_df = get_df(corpus_list, situation_list, sen_type_list, src_type, train_type, class_type)
    eval_neg_df = get_df(corpus_list, negative_list, sen_type_list, src_type, train_type, class_type, positive=False)
    eval_neg_df = eval_neg_df.sample(n=len(eval_df), random_state=0)
    eval_df= pd.concat([eval_df,eval_neg_df]) 
    # eval_df= pd.concat([eval_df,eval_neg_df])

    to_ja_truth = eval_df.loc[eval_df["prefix"].str.contains("mpdd")]["target_text"].tolist()
    to_ja_input = eval_df.loc[eval_df["prefix"].str.contains("mpdd")]["input_text"].tolist()
    to_ja_prefix = eval_df.loc[eval_df["prefix"].str.contains("mpdd")]["prefix"].tolist()

    to_zh_truth = eval_df.loc[eval_df["prefix"].str.contains("cejc")]["target_text"].tolist()
    to_zh_input = eval_df.loc[eval_df["prefix"].str.contains("cejc")]["input_text"].tolist()
    to_zh_prefix = eval_df.loc[eval_df["prefix"].str.contains("cejc")]["prefix"].tolist()

#     to_ja_input = [f"{class_type} classification: " + input_text for input_text in to_ja_input]
#     to_zh_input = [f"{class_type} classification: " + input_text for input_text in to_zh_input]
    to_ja_input = [prefix + ": " + input_text for prefix, input_text in zip(to_ja_prefix, to_ja_input)]
    to_zh_input = [prefix + ": " + input_text for prefix, input_text in zip(to_zh_prefix, to_zh_input)]
    # to_zh_input[:5]
    logging.basicConfig(level=logging.INFO)
    transformers_logger = logging.getLogger("transformers")
    transformers_logger.setLevel(logging.WARNING)

    model_args = T5Args()
    model_args.max_length = 128
    # model_args.length_penalty = 20
    # model_args.num_beams = 10

    model = T5Model("mt5", save_dir+"best_model/", args=model_args, cuda_device=0)
    
    to_ja_preds = model.predict(to_ja_input)
    to_zh_preds = model.predict(to_zh_input)
    
    os.makedirs(save_dir+f'{src_type}/',exist_ok=True)
    
    r_ja_df = pd.DataFrame([to_ja_preds,to_ja_truth],index=[f'{ver_name}', 'truth'])
    r_ja_df.T.to_csv(save_dir+f'{src_type}/ja_preds_truth.csv',encoding='utf_8_sig')

    r_zh_df = pd.DataFrame([to_zh_preds,to_zh_truth],index=[f'{ver_name}', 'truth'])
    r_zh_df.T.to_csv(save_dir+f'{src_type}/zh_preds_truth.csv',encoding='utf_8_sig')